In [ ]:
!pip install openai

In [ ]:
from openai import OpenAI

OPENAI_API_KEY = 'KEY HERE'


client = OpenAI(api_key=OPENAI_API_KEY)


In [ ]:
#upload the JSONL file
with open("JSONL FILE PATH HERE", "rb") as file:
    training_file = client.files.create(
        file=file,
        purpose="fine-tune"
    )

print(f"File uploaded with ID: {training_file.id}")

#create and start the fine-tuning job
job = client.fine_tuning.jobs.create(
    training_file=training_file.id,
    model="gpt-4o-2024-08-06"  #base model to use
)

print(f"Fine-tuning job created: {job.id}")

#status of your fine-tuning job
job_status = client.fine_tuning.jobs.retrieve(job.id)
print(f"Status: {job_status.status}")

In [ ]:
job_status = client.fine_tuning.jobs.retrieve("JOB ID HERE")
print(f"Status: {job_status.status}")

In [ ]:
from openai import OpenAI
import base64

def encode_image_to_base64(image_path):

    """Convert an image file to base64 encoding."""

    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')

def generate_caption_for_image(image_path, model_id):

    """Generate a caption for an image using a fine-tuned model."""

    #encode the image
    base64_image = encode_image_to_base64(image_path)

    #create the API request
    response = client.chat.completions.create(
        model=model_id,  #fine-tuned model id
        messages=[
            {
                "role": "system",
                "content": "You are trained to caption images."
            },
            {
                "role": "user",
                "content": [
                    {
                        "type": "text",
                        "text": "Caption this image."
                    },
                    {
                        "type": "image_url",
                        "image_url": {
                            "url": f"data:image/jpg;base64,{base64_image}"
                        }
                    }
                ]
            }
        ]
    )

    #generated caption
    return response.choices[0].message.content


image_path = "TEST IMAGE HERE"
model_id = "MODEL ID HERE"

caption = generate_caption_for_image(image_path, model_id)
print(f"Generated caption: {caption}")